In [2]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances


# **Chargement du dataset**
def load_dataset(file_path):
    data = pd.read_csv(file_path)
    print(f"Dataset chargé avec {data.shape[0]} lignes et {data.shape[1]} colonnes.")
    print(data.head())
    return data


# **Création d'un graphe basé sur la dissimilarité**
def create_graph_from_dataset(data, features, threshold=0.5):
    # Calcul des distances entre les clients
    distances = euclidean_distances(data[features])
    G = nx.Graph()

    # Ajouter des nœuds (chaque client)
    for i in range(len(data)):
        G.add_node(i, label=f"Client {i}")

    # Ajouter des arêtes basées sur la dissimilarité
    for i in range(len(data)):
        for j in range(i + 1, len(data)):
            if distances[i][j] <= threshold:  # Ajouter une arête si la dissimilarité est en-dessous du seuil
                G.add_edge(i, j, weight=distances[i][j])
    
    return G


# **Affichage des groupes avec couleurs**
def plot_colored_graph(G, partitions):
    pos = nx.spring_layout(G, seed=42)
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'cyan', 'yellow']

    plt.figure(figsize=(10, 10))
    for cluster_id, nodes in partitions.items():
        nx.draw_networkx_nodes(
            G, pos, nodelist=nodes,
            node_color=colors[cluster_id % len(colors)], edgecolors='black'
        )

    nx.draw_networkx_edges(G, pos, alpha=0.5, edge_color='gray')
    nx.draw_networkx_labels(G, pos, font_color='black', font_size=8)
    plt.title("Graphique des clients colorés par groupe (METIS)")
    plt.axis('off')
    plt.show()


# **Partitionnement avec METIS**
def metis_partition(G, k):
    import metis  # Assurez-vous d'avoir installé metis avec `pip install metis`

    # Convertir le graphe NetworkX au format METIS
    edgecuts, parts = metis.part_graph(G, nparts=k)

    # Regrouper les nœuds par partitions
    partitions = {i: [] for i in range(k)}
    for node, part in enumerate(parts):
        partitions[part].append(node)

    return partitions


# **Pipeline de traitement**
def graph_pipeline(file_path, features, threshold=0.5, n_clusters=3):
    # Charger les données
    data = load_dataset(file_path)

    # Créer le graphe
    G = create_graph_from_dataset(data, features, threshold)

    # Partitionnement avec METIS
    partitions = metis_partition(G, n_clusters)

    # Visualiser le graphe
    plot_colored_graph(G, partitions)

    return partitions


# **Exécution**
if __name__ == "__main__":
    # Définir le chemin du fichier et les colonnes d'intérêt
    file_path = 'Mall_Customers.csv'
    features = ['Age', 'Annual Income (k$)', 'Spending Score (1-100)']  # Adapter selon votre dataset

    # Pipeline avec un seuil de dissimilarité et 3 clusters
    partitions = graph_pipeline(file_path, features, threshold=30, n_clusters=3)
    print("Partitions finales :", partitions)


Dataset chargé avec 200 lignes et 5 colonnes.
   CustomerID  Gender  Age  Annual Income (k$)  Spending Score (1-100)
0           1    Male   19                  15                      39
1           2    Male   21                  15                      81
2           3  Female   20                  16                       6
3           4  Female   23                  16                      77
4           5  Female   31                  17                      40


RuntimeError: Could not locate METIS dll. Please set the METIS_DLL environment variable to its full path.